In [2]:
import numpy as np
import scipy as sp
import pandas as pd
import pytraj as pt
import nglview as nv

import matplotlib
from matplotlib import pyplot as plt

import cphmdanalysis as cphmda

In [88]:
nRuns=4
sysName='WT'
with open('pH_List.txt','r') as pHlistFile:
    rep_pH_vals=list(map(float,pHlistFile.readline().split(', ')))
phList=rep_pH_vals*nRuns
nReps=len(rep_pH_vals)
lambdaFileNames=['%s.rep%02g.%02g.lambda'%(sysName,irep+1,irun+1) \
                 for irun in np.arange(nRuns) \
                 for irep in np.arange(len(rep_pH_vals))]
lambdaFiles=[cphmda.lambda_file(filename) for filename in lambdaFileNames]

for lambda_file in lambdaFiles:
    lambda_file.compute_s_values()

pka_data=cphmda.compute_pkas(phList,lambdaFiles) #wrapped_lambda_files)

tempStruc=pt.load('%s.min.rst7'%sysName,top='%s.parm7'%sysName)
orig_struc=pt.load("../../../structures/WT.pdb")

standard_pKa_dict={
    "AS2":3.71,"GL2":4.15,"HIP":6.04,"CYS":8.14,"LYS":10.67}

pKaTable=pd.DataFrame([(tempStruc.top.residue(res-1).name,
 (tempStruc.top.residue(res-1)).original_resid,
 (orig_struc.top.residue(res-1)).original_resid+1,
  "%.3f"%(
      pka_data[ires][2]),
  "%.3f"%(
      pka_data[ires][0])) for ires,res in enumerate(lambdaFiles[0].titrreses)],
            columns=['ResName','Resid','PDB_Resid','Predicted_pKa','Hill_Coef'])
pKaTable["Standard_pKa"]=pKaTable.ResName.apply(
    lambda x: standard_pKa_dict[x])
pKaTable["Delta_pKa"]=pd.Series(pKaTable.Predicted_pKa,dtype=float)- \
    pd.Series(pKaTable.Standard_pKa,dtype=float)
pKaTable.to_csv('pKa_DataTable.csv',index=False)
goodRows=np.sum((pKaTable.Standard_pKa!='nan'))
totalRows=pKaTable.shape[0]
print('Total Number of Residues: ',
      totalRows,
      '\nNumber of succesfully titrated residues: ',
      goodRows,
      '\nNumber of unsuccesfully titrated residues: ',
      totalRows-goodRows)
pKaTable.head()

Total Number of Residues:  95 
Number of succesfully titrated residues:  95 
Number of unsuccesfully titrated residues:  0


/Users/wbotellosmith/anaconda2/envs/py3/lib/python3.7/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,ResName,Resid,PDB_Resid,Predicted_pKa,Hill_Coef,Standard_pKa,Delta_pKa
0,AS2,6,178,2.946,0.718,3.71,-0.764
1,AS2,11,183,3.284,0.583,3.71,-0.426
2,AS2,14,186,3.322,0.766,3.71,-0.388
3,HIP,15,187,6.872,0.860,6.04,0.832
4,CYS,17,189,9.674,0.496,8.14,1.534


In [94]:
for resType in pKaTable.ResName.unique():
    print(pKaTable[(pKaTable.ResName == resType)].dropna().sort_values("Delta_pKa"))

   ResName  Resid  PDB_Resid Predicted_pKa Hill_Coef  Standard_pKa  Delta_pKa
31     AS2    165        337         0.942     0.376          3.71     -2.768
40     AS2    198        370         1.911     0.463          3.71     -1.799
52     AS2    255        427         2.071     0.723          3.71     -1.639
57     AS2    272        444         2.177     1.050          3.71     -1.533
88     AS2    407         81         2.284     0.855          3.71     -1.426
38     AS2    183        355         2.425     0.573          3.71     -1.285
80     AS2    369         43         2.530     0.840          3.71     -1.180
47     AS2    226        398         2.611     0.748          3.71     -1.099
23     AS2    122        294         2.662     0.928          3.71     -1.048
79     AS2    365         39         2.750     2.266          3.71     -0.960
76     AS2    360         34         2.765     0.790          3.71     -0.945
0      AS2      6        178         2.946     0.718          3.